In [ ]:



import os
import glob
import pandas as pd
import numpy as np
from functools import partial
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv1D, MaxPooling1D, Dropout, Dense
from tensorflow.keras.utils import to_categorical, normalize
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.externals import joblib
from PIL import Image
from sklearn import preprocessing
from matplotlib import pyplot
import matplotlib.pyplot as plt



LABELS = {}
counter = iter(range(20))

def pad_and_convert(s):
    """Collect 2000 bytes from packet payload. If payload length is less than
    2000 bytes, pad zeroes at the end. Then convert to integers and normalize."""
    if len(s) < 2000:
        s += '00' * (2000-len(s))
    else:
        s = s[:2000]
    return [float(int(s[i]+s[i+1], 16)/255) for i in range(0, 2000, 2)]

def read_file(f, label):
    df = pd.read_csv(f, index_col=None, header=0)
    df.columns = ['data']
    df['label'] = label
    return df

def preprocess(path):
    files = glob.glob(os.path.join(path, '*.txt'))
    list_ = []
    for f in files:
        label = f.split('/')[-1].split('.')[0]
        LABELS[label] = next(counter)
        labelled_df = partial(read_file, label=LABELS[label])
        list_.append(labelled_df(f))
    df = pd.concat(list_,ignore_index=True)
    return df

def build_model(dropout_rate=0.5, optimizer='adam'):
    activation = 'relu'
    num_classes = len(LABELS)
    model = Sequential()
    model.add(Conv1D(512, strides=2, input_shape=(1000, 1), activation=activation, kernel_size=3, padding='same'))
    model.add(MaxPooling1D())
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(256, strides=2, activation=activation, kernel_size=3, padding='same'))
    model.add(MaxPooling1D())
    model.add(Dropout(dropout_rate))

    model.add(Flatten())
    model.add(Dense(128, activation=activation))
    model.add(Dense(32, activation=activation))

    model.add(Dense(1, activation='sigmoid')) #Using Sigmoid activation for binary classification
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    print(model.summary())
    return model

def main():

    from google.colab import drive
    drive.mount(r"/content/gdrive")
    df = preprocess(path='/content/gdrive/My Drive/Dataset/')

    df['data'] = df['data'].apply(pad_and_convert)
    X_train, X_test, y_train, y_test = train_test_split(df['data'], df['label'],
                                                        test_size=0.3, random_state=4)

    X_train = X_train.apply(pd.Series)
    X_test = X_test.apply(pd.Series)
    X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

    classifier = KerasClassifier(build_fn=build_model)

    parameters = {'batch_size': [32, 64, 128, 256, 512],
                  'epochs': [300],
                  'optimizer': ['adam','rmsdrop', 'adadelta'],
                  'dropout_rate': [0.1, 0.3, 0.5, 0.8]}
    print('calling GridSearchCV\n')
    grid_search = GridSearchCV(estimator=classifier,
                               param_grid=parameters,
                               scoring='accuracy',
                               cv=3, verbose=10)

    grid_result = grid_search.fit(X_train, y_train)
    best_parameters = grid_result.best_params_
    best_accuracy = grid_result.best_score_

    print('Best parameters are: {}\nBest score is: {}\n'.format(best_parameters, best_accuracy))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("printing .... %f (%f) with: %r\n" % (mean, stdev, param))
    
    print(grid_result.best_params_)
    grid_predictions  = grid_result.predict(X_test)
    print('classificationÂ report\n')
    print(classification_report(y_test,grid_predictions))
   


if __name__ == '__main__':
    main()

